#### Copyright IBM All Rights Reserved.

# RestAPI - SQL queries

This notebook includes basic Python examples for the Db2 REST API.

The official documentation of the DB2 RestAPI is here:
https://cloud.ibm.com/apidocs/db2-on-cloud/db2-on-cloud-v4



## Prerequisites:
1. Python 3.6 and above
2. Db2 instance (on premises or on cloud)
3. Data already loaded in your Db2 instance
4. Db2 connection credentials on hand

## Required libraries

To run the examples in this notebook, you need to install the following libraries:

- `http.client`: client side of the HTTP and HTTPS protocol
- `json`: JSON encoder and decoder
- `pandas`: For data manipulation and analysis
- `matplotlib.pyplot`: For command style functions that make matplotlib work like MATLAB
- `requests`: For making HTTP calls
- `time`: For various time-related functions

In [1]:
import http.client
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
from pandas.io.json import json_normalize

Database credentials can be defined in the file `db2_connection.ipynb`. Import this file to use the function `get_credentials_for_db()`.

In [2]:
%run db2_connection.ipynb

## Step 1 - Generating an Access Token
The first step in generating an access token is to build the URL for the service. From a programming perspective, the host URL will be combined with the API library to form the stub for all future RESTful URL requests. The HOST IP address is derived from the Db2 connect parameters.

If available, copy your service credentials provided in JSON format into the curly brackets. The definition of db2id should look like this:

```json
db2id = {
  "db": "myDatabaseName",
  "host": "anydbservice.mycloud.net",
  "hostname": "anydbservice.mycloud.net",
  "https_url": "https://anydbservice.mycloud.net",
  "port": 50000,
  "userid" : "xyz12345",
  "password" :"secretPassword"
}
```

In [ ]:
db2id = {
}

In [3]:
#db2id = get_credentials_for_db('Db2-lite')
db2id = get_credentials_for_db('Db2-flex')
#db2id = get_credentials_for_db('Db2-WoC')

In [ ]:
# Test
db2id['hostname']

Define the connection and its parameters

In [4]:
conn = http.client.HTTPSConnection(db2id['hostname'])
payload = "{\"userid\": \"" + db2id['username'] + "\" ,\"password\": \"" + db2id['password'] + "\" }"
headers = { 'content-type': "application/json" }

Call the method `auth/tokens` via RestAPI

In [5]:
conn.request("POST", "/dbapi/v4/auth/tokens", payload, headers)

res = conn.getresponse()
data = res.read()
# print(data.decode("utf-8"))
print ("Status: " + str(res.status) + ", Reason: " + str(res.reason))

result = json.loads(data)

Status: 200, Reason: OK


Format the output data 

In [6]:
token = result['token']
headers = {
    'content-type': "application/json",
    'authorization': "Bearer " + token 
    }

In [ ]:
# print token
token

## Step 2 - Execute SQL Queries
The service
```javascript   
/sql_jobs
```
executes one or more SQL statements as a background job. This endpoint returns a `job ID`
 that can be used to retrieve the results.

In [7]:
SQLQuery = "select * from BLUADMIN.PERSON order by zip"
payload = "{\"commands\":\"" + SQLQuery + "\", \
            \"limit\":12, \
            \"separator\":\";\", \
            \"stop_on_error\":\"no\" \
            }"

In [8]:
conn.request("POST", "/dbapi/v4/sql_jobs", payload, headers)

res = conn.getresponse()
data = res.read()

#Decode result and keep them in a dictionary
data_dict = json.loads(data)

In [ ]:
data_dict

In [9]:
# job ID
data_dict['id']

'1594823404634_105175544'

## Step 3 - Fetches partial results of a SQL job execution
The service
```javascript  
    /sql_jobs/{id}
```
returns the current status of a SQL job execution along with any results of SQL statements that have already been executed. Clients are supposed to poll this endpoint until the status returned is either 'completed', which indicates all SQL statements have completed executing, or 'failed', which indicates the job failed to execute and therefore is considered terminated. The returned list of results is not cumulative. That means, results that were fetched in a previous call will not be returned again, only new results (i.e. that were not fetched yet) will be included. For example, assuming a job with 10 SQL statements, the first call returns status "running" and 6 results, the second call returns status "running" and an empty list of results, a third call status "completed" and 4 results. Any subsequent calls would return status "completed" and an empty list of results.

In [10]:
conn.request("GET", "/dbapi/v4/sql_jobs/" + data_dict['id'], payload, headers)

res = conn.getresponse()
data = res.read()
data_dict = json.loads(data)

In [11]:
pd.DataFrame(data_dict['results'][0]['rows'], columns=data_dict['results'][0]['columns'])

,ID,FULLNAME,PID,CITY,ZIP,EMAIL,PHONE
0,10032,Victor,16110428 9838,Telfs,01318,Praesent@ornareplacerat.org,(037509) 763628
1,10043,Porter,16360511 3210,Aurora,02198,ac.mattis.ornare@consequatenim.edu,(087) 57258900
2,10041,Lucian,16290710 3846,Rio nell'Elba,034283,Ut.sagittis@Namnulla.com,(0604) 75851251
3,10057,Nicholas,16440705 9304,Montleban,0346,mauris.erat.eget@Inmi.net,(030773) 848041
4,10090,Blaze,16341027 8570,Lipetsk,06251-679,amet.luctus.vulputate@luctusipsum.net,(0019) 29768342
5,10049,Forrest,16110102 4121,Solihull,09795,eu.erat@vestibulumnec.co.uk,(08406) 0057715
6,10019,Mufutau,16320214 0673,Taber,09861,Aliquam@lobortisrisusIn.org,(037173) 477132
7,10004,Harper,16840425 2283,Cervino,10605,auctor.ullamcorper.nisl@vitae.co.uk,(032653) 671059
8,10034,Cade,16340629 9986,Torgny,108460,eu.tempor@AliquamnislNulla.co.uk,(0686) 94256770
9,10098,Samson,16110409 3883,Annapolis County,12-957,metus.facilisis@euismodetcommodo.org,(03860) 9960362


### Alternative: Query table data (step2 + 3)
The service
```javascript  
    /admin/schemas/{schema_name}/tables/{table_name}/data
```

fetches the table data up to a maximum of 100,000 rows. Currently it's not possible to retrieve data from tables that contain CLOB, BLOB or DBCLOB values.

In [12]:
service = "/dbapi/v4/admin/schemas/"  \
        + "BLUADMIN" \
        + "/tables/" \
        + "PERSON"   \
        + "/data?rows_return=12"
# service

In [13]:
conn.request("GET", service, headers=headers)
res = conn.getresponse()
data = res.read()

#Decode result and keep them in a dictionary
result_dict = json.loads(data)

In [14]:
column_names = [ sub['name'] for sub in result_dict['columns'] ] 
pd.DataFrame(result_dict['rows'], columns=column_names)

,ID,FULLNAME,PID,CITY,ZIP,EMAIL,PHONE
0,10000,Erich,16560208 0060,Cabildo,74208-82216,Ut.nec@enim.co.uk,(03240) 3433625
1,10001,Shad,16420717 2661,Nyandoma,781552,tincidunt.orci.quis@metuseu.edu,(032965) 431180
2,10002,Alec,16121101 2206,Grimbergen,56736,tempus.non@famesacturpis.ca,(0111) 87808412
3,10003,Paki,16970123 6235,Soma,44-067,metus.vitae@Nunc.edu,(026) 56243189
4,10004,Harper,16840425 2283,Cervino,10605,auctor.ullamcorper.nisl@vitae.co.uk,(032653) 671059
5,10005,Ethan,16120910 7497,Great Falls,70790,ultrices.posuere@egetvarius.com,(0996) 56248753
6,10006,Yoshio,16001107 6155,Villafalletto,56938,nec.tempus.scelerisque@ante.ca,(035920) 423116
7,10007,Noah,16311227 4521,Heikruis,61432,velit@semsemper.edu,(04174) 6862313
8,10008,Matthew,16750802 2485,Castelbaldo,65-388,ridiculus.mus.Donec@infaucibus.co.uk,(0487) 79520636
9,10009,Amal,16301015 7927,Dawson Creek,97990,fermentum.convallis.ligula@orci.ca,(04683) 8863010


## Summary
Using RESTful calls to Db2 removes much of the complexity of communicating with the database. There are no drivers required, no configuration file, nor any administration required on the client that is communicating with the database. All communication is done using RESTful API calls, which are available on all browsers and all operating systems. The calls to the database are replaced with standard POST and GET requests. Enabling RESTful support to Db2 opens up the type of applications that you can write and clients that you can connect to Db2 with.

#### Credits:  IBM 2020, Stefan Hummel [stefan.hummel@de.ibm.com]